# **Downloads and Imports**

In [ ]:
!pip install pennylane

In [ ]:
# Imports
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp

/usr/local/lib/python3.12/dist-packages/pennylane/__init__.py:209: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.6.2. You have version 0.7.2 installed. Please downgrade JAX to 0.6.2 to avoid runtime errors using python -m pip install jax~=0.6.0 jaxlib~=0.6.0
  warnings.warn(


# **Classical Reset Gate**

In [ ]:
input_dim = 5
hidden_dim = 10
np.random.seed(42)

print(f"Input dimension (x_t): {input_dim}")
print(f"Hidden dimension (h_t, r_t): {hidden_dim}")

Input dimension (x_t): 5
Hidden dimension (h_t, r_t): 10


In [ ]:
def sigmoid(z):
  return 1 / (1 + np.exp(-z))

test_input = np.array([-10, 0, 10])
print(f"\nSigmoid test input: {test_input}")
print(f"Sigmoid test output: {sigmoid(test_input)}")


Sigmoid test input: [-10   0  10]
Sigmoid test output: [4.53978687e-05 5.00000000e-01 9.99954602e-01]


In [ ]:
def initialize_data_and_weights(h_dim, i_dim):

    h_prev = np.random.rand(1, h_dim)
    x_t = np.random.rand(1, i_dim)
    W_hr = np.random.rand(h_dim, h_dim)
    W_xr = np.random.rand(i_dim, h_dim)
    W_r = np.vstack((W_hr, W_xr))
    return h_prev, x_t, W_hr, W_xr, W_r

h_prev, x_t, W_hr, W_xr, W_r = initialize_data_and_weights(hidden_dim, input_dim)

print("--- Vector Shapes ---")
print(f"h_prev shape: {h_prev.shape}")
print(f"x_t shape:    {x_t.shape}")

print("\n--- Weight Matrix Shapes ---")
print(f"W_hr shape (Method 2): {W_hr.shape}")
print(f"W_xr shape (Method 2): {W_xr.shape}")
print(f"W_r shape (Method 1):  {W_r.shape}")

expected_shape = (hidden_dim + input_dim, hidden_dim)
assert W_r.shape == expected_shape
print(f"\nVerification successful: W_r shape is {expected_shape}")

--- Vector Shapes ---
h_prev shape: (1, 10)
x_t shape:    (1, 5)

--- Weight Matrix Shapes ---
W_hr shape (Method 2): (10, 10)
W_xr shape (Method 2): (5, 10)
W_r shape (Method 1):  (15, 10)

Verification successful: W_r shape is (15, 10)


In [ ]:
def compute_reset_gate_concatenated(h_prev, x_t, W_r):
    """
    Computes the reset gate using the concatenated method.
    Formula: r_t = sigma(W_r . [h_{t-1}, x_t])
    """

    combined_input = np.concatenate((h_prev, x_t), axis=1)
    z = np.dot(combined_input, W_r)
    r_t = sigmoid(z)
    return r_t

result_1 = compute_reset_gate_concatenated(h_prev, x_t, W_r)

print("--- Method 1 (Concatenated) ---")
print(f"Resulting r_t shape: {result_1.shape}")
print(f"Result (first 5 elements): {result_1[0, :5]}")

--- Method 1 (Concatenated) ---
Resulting r_t shape: (1, 10)
Result (first 5 elements): [0.97294525 0.97136796 0.98581977 0.94884454 0.94442912]


In [ ]:
def compute_reset_gate_separate(h_prev, x_t, W_hr, W_xr):
    """
    Computes the reset gate using the separate matrices method.
    Formula: r_t = sigma(W_hr . h_{t-1} + W_xr . x_t)
    """

    term_h = np.dot(h_prev, W_hr)
    term_x = np.dot(x_t, W_xr)
    z = term_h + term_x
    r_t = sigmoid(z)
    return r_t

result_2 = compute_reset_gate_separate(h_prev, x_t, W_hr, W_xr)

print("--- Method 2 (Separate) ---")
print(f"Resulting r_t shape: {result_2.shape}")
print(f"Result (first 5 elements): {result_2[0, :5]}")

--- Method 2 (Separate) ---
Resulting r_t shape: (1, 10)
Result (first 5 elements): [0.97294525 0.97136796 0.98581977 0.94884454 0.94442912]


In [ ]:
print("--- Verification ---")
print(f"Method 1 (first 5): {result_1[0, :5]}")
print(f"Method 2 (first 5): {result_2[0, :5]}")

are_identical = np.allclose(result_1, result_2)
identical = "Identical" if are_identical else "Not identical"
print(f"Results are {identical}")
print(f"Results are numerically {identical}")
print()

# **Quantum Reset Gate**

In [ ]:
h_qubits = hidden_dim
x_qubits = input_dim
total_qubits = h_qubits + x_qubits

h_wires = list(range(h_qubits))
x_wires = list(range(h_qubits, total_qubits))

print(f"\n--- Quantum Setup ---")
print(f"Hidden state qubits (h): {h_qubits} (Wires: {h_wires[0]}...{h_wires[-1]})")
print(f"Input vector qubits (x): {x_qubits} (Wires: {x_wires[0]}...{x_wires[-1]})")
print(f"Total qubits required: {total_qubits}")

dev = qml.device("default.qubit", wires=total_qubits)


--- Quantum Setup ---
Hidden state qubits (h): 10 (Wires: 0...9)
Input vector qubits (x): 5 (Wires: 10...14)
Total qubits required: 15


In [ ]:
def encode_inputs(h_vec, x_vec):
    """
    Encodes the classical h and x vectors into the quantum circuit
    using angle embedding (Ry rotations).

    Args:
        h_vec (pnp.array): The 1D hidden state vector.
        x_vec (pnp.array): The 1D input vector.
    """
    qml.AngleEmbedding(h_vec, wires=h_wires, rotation='Y')
    qml.AngleEmbedding(x_vec, wires=x_wires, rotation='Y')

@qml.qnode(dev)
def test_encoding_circuit(h_vec, x_vec):
    """A simple QNode to visualize the encoding layer."""
    encode_inputs(h_vec, x_vec)
    return qml.state()

h_prev_pnp = pnp.array(h_prev[0], requires_grad=False)
x_t_pnp = pnp.array(x_t[0], requires_grad=False)

print("--- Quantum Data Encoding (AngleEmbedding) ---")
drawer = qml.draw(test_encoding_circuit)(h_prev_pnp, x_t_pnp)
print(drawer)

--- Quantum Data Encoding (AngleEmbedding) ---
 0: ─╭AngleEmbedding(M0)─┤  State
 1: ─├AngleEmbedding(M0)─┤  State
 2: ─├AngleEmbedding(M0)─┤  State
 3: ─├AngleEmbedding(M0)─┤  State
 4: ─├AngleEmbedding(M0)─┤  State
 5: ─├AngleEmbedding(M0)─┤  State
 6: ─├AngleEmbedding(M0)─┤  State
 7: ─├AngleEmbedding(M0)─┤  State
 8: ─├AngleEmbedding(M0)─┤  State
 9: ─╰AngleEmbedding(M0)─┤  State
10: ─╭AngleEmbedding(M1)─┤  State
11: ─├AngleEmbedding(M1)─┤  State
12: ─├AngleEmbedding(M1)─┤  State
13: ─├AngleEmbedding(M1)─┤  State
14: ─╰AngleEmbedding(M1)─┤  State

M0 = 
[0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
 0.05808361 0.86617615 0.60111501 0.70807258]
M1 = 
[0.02058449 0.96990985 0.83244264 0.21233911 0.18182497]


In [ ]:
def variational_layer(weights):
    """
    The trainable part of the circuit ("quantum weights").
    It applies parameterized rotations and entanglement.

    Structure:
    1. Parameterized Rotations (Ry, Rz) on all qubits.
    2. Internal Entanglement (CNOTs within h and within x).
    3. External Entanglement (CNOTs between h and x).

    Args:
        weights (pnp.array): A 1D array of trainable parameters.
    """

    num_rotations = 2
    weights_per_layer = num_rotations * total_qubits

    if weights.shape[0] != weights_per_layer:
        raise ValueError(
            f"Expected {weights_per_layer} weights, but got {weights.shape[0]}"
        )

    weights_ry = weights[:total_qubits]
    weights_rz = weights[total_qubits:]

    for i in range(total_qubits):
        qml.RY(weights_ry[i], wires=i)
        qml.RZ(weights_rz[i], wires=i)

    for i in range(h_qubits - 1):
        qml.CNOT(wires=[h_wires[i], h_wires[i + 1]])

    if x_qubits > 1:
        for i in range(x_qubits - 1):
            qml.CNOT(wires=[x_wires[i], x_wires[i + 1]])

    for i in range(h_qubits):
        h_wire = h_wires[i]
        x_wire = x_wires[i % x_qubits]
        qml.CNOT(wires=[h_wire, x_wire])

@qml.qnode(dev)
def test_variational_circuit():
    """A simple QNode to visualize the variational layer."""
    num_params = 2 * total_qubits
    dummy_weights = pnp.array([0.1] * num_params, requires_grad=True)

    variational_layer(dummy_weights)
    return qml.state()

print("--- Variational Circuit Structure ---")
drawer = qml.draw(test_variational_circuit)()
print(drawer)

print(f"\nThis layer requires {2 * total_qubits} trainable parameters.")

--- Variational Circuit Structure ---
 0: ──RY(0.10)──RZ(0.10)─╭●─────────────────────────╭●────────────────────────────┤  State
 1: ──RY(0.10)──RZ(0.10)─╰X─╭●──────────────────────│──╭●─────────────────────────┤  State
 2: ──RY(0.10)──RZ(0.10)────╰X─╭●───────────────────│──│──╭●──────────────────────┤  State
 3: ──RY(0.10)──RZ(0.10)───────╰X─╭●────────────────│──│──│──╭●───────────────────┤  State
 4: ──RY(0.10)──RZ(0.10)──────────╰X─╭●─────────────│──│──│──│──╭●────────────────┤  State
 5: ──RY(0.10)──RZ(0.10)─────────────╰X─╭●──────────│──│──│──│──│──╭●─────────────┤  State
 6: ──RY(0.10)──RZ(0.10)────────────────╰X─╭●───────│──│──│──│──│──│──╭●──────────┤  State
 7: ──RY(0.10)──RZ(0.10)───────────────────╰X─╭●────│──│──│──│──│──│──│──╭●───────┤  State
 8: ──RY(0.10)──RZ(0.10)──────────────────────╰X─╭●─│──│──│──│──│──│──│──│──╭●────┤  State
 9: ──RY(0.10)──RZ(0.10)─────────────────────────╰X─│──│──│──│──│──│──│──│──│──╭●─┤  State
10: ──RY(0.10)──RZ(0.10)─╭●─────────────────────────

In [ ]:
def define_observables():
    """
    Defines the measurement observables.

    We measure the PauliZ expectation value for each of the h_qubits.
    The result will be a vector of shape (hidden_dim,).
    """
    return [qml.expval(qml.PauliZ(i)) for i in h_wires]

obs_list = define_observables()

print("--- Measurement Observables ---")
print(f"Number of observables defined: {len(obs_list)}")
print("Example observables:")
print(obs_list[0])
if len(obs_list) > 1:
    print(obs_list[1])

--- Measurement Observables ---
Number of observables defined: 10
Example observables:
expval(Z(0))
expval(Z(1))


In [ ]:
@qml.qnode(dev)
def quantum_reset_gate(inputs, weights):
    """
    The full QNode for the reset gate's linear transformation.

    1. Encodes classical inputs (h, x).
    2. Applies the trainable variational layer.
    3. Returns the expectation values (z) as a single stacked array.

    Args:
        inputs (pnp.array): A 1D array containing h_prev and x_t concatenated.
        weights (pnp.array): A 1D array of trainable parameters.

    Returns:
        pnp.array: A 1D array of expectation values (shape [hidden_dim,]).
    """

    h_vec = inputs[:h_qubits]
    x_vec = inputs[h_qubits:]
    encode_inputs(h_vec, x_vec)
    variational_layer(weights)
    return qml.math.stack(define_observables())

num_params = 2 * total_qubits
test_weights = pnp.array(np.random.rand(num_params), requires_grad=True)
test_input_vector = pnp.concatenate([h_prev_pnp, x_t_pnp])

print("--- Full Quantum Model (QNode) Test ---")
print(f"Input vector shape: {test_input_vector.shape}")
print(f"Weights vector shape: {test_weights.shape}")

z_quantum = quantum_reset_gate(test_input_vector, test_weights)

print(f"\nQuantum output (z) shape: {z_quantum.shape}")
print(f"Quantum output (z) example: {z_quantum}")

drawer = qml.draw(quantum_reset_gate)(test_input_vector, test_weights)
print("\n--- Full Circuit Diagram ---")
print(drawer)

--- Full Quantum Model (QNode) Test ---
Input vector shape: (15,)
Weights vector shape: (30,)

Quantum output (z) shape: (10,)
Quantum output (z) example: [0.76784862 0.32257878 0.23096004 0.08593475 0.0577677  0.05690953
 0.04790677 0.02204261 0.00702813 0.00446476]

--- Full Circuit Diagram ---
 0: ─╭AngleEmbedding(M0)──RY(0.32)──RZ(0.11)─╭●─────────────────────────╭●───────────────────── ···
 1: ─├AngleEmbedding(M0)──RY(0.19)──RZ(0.92)─╰X─╭●──────────────────────│──╭●────────────────── ···
 2: ─├AngleEmbedding(M0)──RY(0.04)──RZ(0.88)────╰X─╭●───────────────────│──│──╭●─────────────── ···
 3: ─├AngleEmbedding(M0)──RY(0.59)──RZ(0.26)───────╰X─╭●────────────────│──│──│──╭●──────────── ···
 4: ─├AngleEmbedding(M0)──RY(0.68)──RZ(0.66)──────────╰X─╭●─────────────│──│──│──│──╭●───────── ···
 5: ─├AngleEmbedding(M0)──RY(0.02)──RZ(0.82)─────────────╰X─╭●──────────│──│──│──│──│──╭●────── ···
 6: ─├AngleEmbedding(M0)──RY(0.51)──RZ(0.56)────────────────╰X─╭●───────│──│──│──│──│──│──╭●─── ···
 7

In [ ]:
def hybrid_model(inputs, weights):
    """
    The full hybrid model.
    1. Runs the QNode to get the linear output z.
    2. Applies the classical sigmoid function.
    """
    z_quantum = quantum_reset_gate(inputs, weights)
    r_t_quantum = 1 / (1 + pnp.exp(-z_quantum))
    return r_t_quantum

def create_training_data(num_samples):
    """
    Generates a training dataset using the classical Method 2.
    """
    X_train_list = []
    Y_train_list = []

    for _ in range(num_samples):
        h_sample = np.random.rand(1, hidden_dim)
        x_sample = np.random.rand(1, input_dim)

        y_target = compute_reset_gate_separate(h_sample, x_sample, W_hr, W_xr)

        x_input = pnp.concatenate([
            pnp.array(h_sample[0]),
            pnp.array(x_sample[0])
        ], requires_grad=False)

        y_target_pnp = pnp.array(y_target[0], requires_grad=False)

        X_train_list.append(x_input)
        Y_train_list.append(y_target_pnp)

    X_train = pnp.stack(X_train_list)
    Y_train = pnp.stack(Y_train_list)

    return X_train, Y_train

def cost_function(weights, X_batch, Y_batch):
    """
    Mean Squared Error (MSE) loss function.
    This version iterates over the batch, which is compatible
    with the autograd interface.
    """
    total_squared_error = 0.0
    num_samples = X_batch.shape[0]

    if num_samples == 0:
        return 0.0

    num_outputs = Y_batch.shape[1]

    for i in range(num_samples):
        x = X_batch[i]
        y_true = Y_batch[i]
        y_pred = hybrid_model(x, weights)

        squared_error = (y_pred - y_true)**2
        total_squared_error = total_squared_error + pnp.sum(squared_error)

    return total_squared_error / (num_samples * num_outputs)

num_params = 2 * total_qubits
num_train_samples = 50
learning_rate = 0.01
num_epochs = 300

params = pnp.array(np.random.rand(num_params), requires_grad=True)

X_data, Y_data = create_training_data(num_train_samples)
print(f"Created training data:")
print(f"X_data shape: {X_data.shape}")
print(f"Y_data shape: {Y_data.shape}")

opt = qml.AdamOptimizer(stepsize=learning_rate)

print("\n--- Starting Training ---")
for epoch in range(num_epochs + 1):
    params = opt.step(cost_function, params, X_batch=X_data, Y_batch=Y_data)

    if epoch % 20 == 0:
        current_loss = cost_function(params, X_data, Y_data)
        print(f"Epoch {epoch:3d} | Loss: {current_loss:.6f}")

final_trained_params = params
print("--- Training Complete ---")

Created training data:
X_data shape: (50, 15)
Y_data shape: (50, 10)

--- Starting Training ---
Epoch   0 | Loss: 0.189655
Epoch  20 | Loss: 0.169820
Epoch  40 | Loss: 0.138765
Epoch  60 | Loss: 0.105556
Epoch  80 | Loss: 0.086342
Epoch 100 | Loss: 0.079833
Epoch 120 | Loss: 0.078429
Epoch 140 | Loss: 0.078251
Epoch 160 | Loss: 0.078238
Epoch 180 | Loss: 0.078237
Epoch 200 | Loss: 0.078237
Epoch 220 | Loss: 0.078237
Epoch 240 | Loss: 0.078237
Epoch 260 | Loss: 0.078237
Epoch 280 | Loss: 0.078237
Epoch 300 | Loss: 0.078237
--- Training Complete ---


# **Comparing Classical and Quantum Gates**

In [ ]:
def compare_models(h_vec_np, x_vec_np, trained_weights):
    """
    Compares the classical and quantum model outputs for a single sample.
    """
    print("--- Final Model Comparison ---")

    h_classical_in = h_vec_np.reshape(1, hidden_dim)
    x_classical_in = x_vec_np.reshape(1, input_dim)

    classical_output = compute_reset_gate_separate(
        h_classical_in,
        x_classical_in,
        W_hr,
        W_xr
    )

    h_pnp = pnp.array(h_vec_np, requires_grad=False)
    x_pnp = pnp.array(x_vec_np, requires_grad=False)
    quantum_input = pnp.concatenate([h_pnp, x_pnp])
    quantum_output = hybrid_model(quantum_input, trained_weights)
    classical_flat = classical_output.flatten()
    quantum_flat = quantum_output

    print(f"{'Index':<7} | {'Classical (Target)':<20} | {'Quantum (Predicted)':<20} | {'Difference':<20}")
    print("-" * 71)

    total_diff = 0.0
    for i in range(len(classical_flat)):
        diff = np.abs(classical_flat[i] - quantum_flat[i])
        total_diff += diff
        print(f"{i:<7} | {classical_flat[i]:<20.6f} | {quantum_flat[i]:<20.6f} | {diff:<20.6f}")

    print("-" * 71)
    print(f"Mean Absolute Difference: {total_diff / len(classical_flat):.6f}")

h_test = np.random.rand(hidden_dim)
x_test = np.random.rand(input_dim)
compare_models(h_test, x_test, final_trained_params)

--- Final Model Comparison ---
Index   | Classical (Target)   | Quantum (Predicted)  | Difference          
-----------------------------------------------------------------------
0       | 0.966636             | 0.698078             | 0.268558            
1       | 0.957815             | 0.697415             | 0.260400            
2       | 0.978210             | 0.687026             | 0.291184            
3       | 0.933124             | 0.687017             | 0.246107            
4       | 0.959554             | 0.686772             | 0.272782            
5       | 0.984959             | 0.679143             | 0.305816            
6       | 0.942162             | 0.668268             | 0.273894            
7       | 0.942213             | 0.667990             | 0.274224            
8       | 0.975117             | 0.656438             | 0.318679            
9       | 0.980009             | 0.653381             | 0.326627            
--------------------------------------------------